The model in this directory continues to run the model vasiliev24_L3M11_extremeperi in isolation to understand how the system is in or out of equilibrium

In [ ]:
using CairoMakie
using LilGuys
using Arya

## Verifying vcirc method

In [ ]:
snap = Snapshot("combined.hdf5/1")

In [ ]:
r = calc_r(snap)
v = calc_v(snap)
phi = snap.Φs
m = snap.masses

idx = sortperm(r)
r = r[idx]
v = v[idx]
phi = phi[idx]
m = m[idx]

In [ ]:
v_circ_all = calc_v_circ.(r, cumsum(m))

In [ ]:
phi_1 = LilGuys.calc_radial_discrete_Φ(r, m)
eps_1 = -1/2 * v .^ 2 .- phi_1
filt_1 = eps_1 .> 0
println(sum(.!filt_1))

dN = sum(.!filt_1)
phi_2 = phi_1
while dN > 0
    phi_2 = LilGuys.calc_radial_discrete_Φ(r[filt_1], m[filt_1])
    eps_2 = -1/2 * v[filt_1] .^ 2 .- phi_2
    filt_2 = eps_2 .> 0
    println(sum(.!filt_2))
    
    filt_1[eachindex(r)[filt_1][.!filt_2]] .= false
    dN = sum(.!filt_2)
end


In [ ]:
v_circ_bound = calc_v_circ.(r[filt_1], cumsum(m[filt_1]))

In [ ]:
LilGuys.get_bound_recursive_1D(snap)

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r",
    ylabel = "log v",
    )

skip = 100
lines!(log10.(r[1:skip:end]), log10.(v_circ_all[1:skip:end]))
lines!(log10.(r[filt_1][1:skip:end]), log10.(v_circ_bound[1:skip:end]))

r1, v1, _ = LilGuys.calc_v_circ(snap; skip=skip)
lines!(log10.(r1), log10.(v1), linestyle=:dot)


r1, v1, _ = LilGuys.calc_v_circ(snap; skip=skip, filter_bound=:simple)
lines!(log10.(r1), log10.(v1), linestyle=:dot)


r1, v1, _ = LilGuys.calc_v_circ(snap; filter_bound=:false, skip=skip)
lines!(log10.(r1), log10.(v1), linestyle=:dot)
fig

In [ ]:
scatter(phi[1:100:end], phi_1[1:100:end])

In [ ]:
scatter(phi[filt_1][1:100:end], phi_2[1:100:end])

## Profiles

In [ ]:
function load_profiles(filename)
    profiles = LilGuys.read_structs_from_hdf5(filename, LilGuys.MassProfile3D)

	snap_idx = parse.(Int, first.(profiles))

	profiles = last.(profiles)

	profiles = profiles[sortperm(snap_idx)]
	snap_idx = sort(snap_idx)

    return profiles
end

In [ ]:
profs_cont = load_profiles("profiles.hdf5");

In [ ]:
save = Makie.save

In [ ]:
figdir = "figures"

In [ ]:
profs_orbit =  load_profiles("../vasiliev24_L3M11_extremeperi/profiles.hdf5");

In [ ]:
profiles = vcat(profs_orbit, profs_cont);

In [ ]:
r_max = [f.r_circ_max for f in profiles]
v_max = [f.v_circ_max for f in profiles]

In [ ]:
prof_i = profiles[1]
prof_f = profs_orbit[end]
prof_fc = profiles[end]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel=L"\log \; r_\textrm{circ}\ /\ \textrm{kpc}", 
    ylabel=L"$v_\textrm{circ}$ / km s$^{-1}$",
    yscale=log10,
    yticks=[1, 10, 20, 30, 40, 50, 60],
    yminorticks=[1:9; 10:2:60],
    limits=((-1, 2.5), (10, 35)),
    xgridvisible=false,
    ygridvisible=false
)


lines!(log10.(prof_i.r_circ), prof_i.v_circ * V2KMS, label="initial")

lines!(log10.(prof_f.r_circ), prof_f.v_circ * V2KMS, label="final")
lines!(log10.(prof_fc.r_circ), prof_fc.v_circ * V2KMS, label="final+iso")


α = 0.4
β = 0.65
x = LinRange(1, 0.1, 100)

y = @. 2^α * x^β * (1 + x^2)^(-α)
lines!(log10.(x .* r_max[1]), y .* v_max[1] * V2KMS,  label="EN21",
color=:black, linestyle=:dash)

scatter!(log10.(r_max), v_max * V2KMS, color=Arya.COLORS[4], label=L"v_\textrm{circ,\ max}", markersize=3)

    
axislegend(ax, position=:rt)

save(figdir * "/vcirc_rcirc_cont.pdf", fig)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel=L"v_\text{circ} / \text{km\,s^{-1}}",
limits=(nothing, (nothing, nothing))
)
x = [prof.time for prof in profiles]
lines!(x*T2GYR, v_max * V2KMS, label=L"maximum $v_\text{circ}$")
#scatter!(x, v_h, label=L"r=r_h")
#axislegend(ax)


fig

![](figures/boundmass.pdf)

# Summary

In this model, the stellar component is not as strongly affected as we might expect. 
Is this because the feature is fully mixed because of the relaxation / dynamical timescales, so the
tidal bump just does not occur in inner regions? Do we need higher resolution?
One complication is that the LMC does not have stars, and likely passing through the centre means that a DM only 
simulation causes several limitations

- dynamical friction is neglected
- captured and lost stars are neglected
- do gas clouds affect stellar motions? no?
- non-spherical potential in centre (barred irregular galaxy!)

Additionally, we may be looking for features where numerical resolution may not be high enough. 
